<a href="https://colab.research.google.com/github/MajidNoorani/depression_tweet_transformer/blob/main/depression_tweet_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# Load the dataset
dataset = load_dataset("ziq/depression_tweet")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 27613
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 9204
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9205
    })
})

In [4]:
df_train = pd.DataFrame({
    'text': dataset['train']['text'],
    'label': dataset['train']['label']
})

df_test = pd.DataFrame({
    'text': dataset['test']['text'],
    'label': dataset['test']['label']
})

df_val = pd.DataFrame({
    'text': dataset['validation']['text'],
    'label': dataset['validation']['label']
})

In [5]:
df_train['label'].value_counts()

0    15655
1    11958
Name: label, dtype: int64

In [6]:
max_words_length = df_train['text'].apply(lambda x: len(str(x).split())).max()

print("Maximum number of words in text:", max_words_length)

max_words_index = df_train['text'].apply(lambda x: len(str(x).split())).idxmax()

print("Index of the row with maximum number of words:", max_words_index)

Maximum number of words in text: 4239
Index of the row with maximum number of words: 1587


In [7]:
sorted(df_train['text'].apply(lambda x: len(str(x).split())), reverse=True)[:10]

[4239, 2676, 2081, 1796, 1656, 1653, 1559, 1537, 1470, 1462]

In [8]:
vocab_size = len(set(" ".join(df_train['text'].astype(str)).split()))
vocab_size

76895

In [9]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [10]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [11]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
                num_words=None,
                filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                lower=True,
                split=' ',
                char_level=False,
                oov_token=None,
                analyzer=None,
            )

In [12]:
tokenizer.fit_on_texts(df_train['text'].values)

In [13]:
vocab_size = 70000  # Only consider the top vocab_size words
maxlen = 2000  # Only consider the first maxlen words of each text
# (x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(df_train['text']), "Training sequences")
print(len(df_train['label']), "Validation sequences")
x_train = tokenizer.texts_to_sequences(df_train['text'].values)
x_val = tokenizer.texts_to_sequences(df_val['text'].values)
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen, padding='post', truncating='post')

27613 Training sequences
27613 Validation sequences


In [14]:
len(x_train)

27613

In [15]:
embed_dim = 128  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [16]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.keras", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=3,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=True,
        start_from_epoch=0
    )
    ]

In [18]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, df_train['label'], batch_size=64, epochs=20, validation_data=(x_val, df_val['label']), callbacks=callbacks
)

Epoch 1/20
432/432 [==============================] - 251s 565ms/step - loss: 0.4370 - accuracy: 0.7641 - val_loss: 0.2283 - val_accuracy: 0.9170
Epoch 2/20
432/432 [==============================] - 252s 582ms/step - loss: 0.1884 - accuracy: 0.9348 - val_loss: 0.1888 - val_accuracy: 0.9331
Epoch 3/20
432/432 [==============================] - 236s 546ms/step - loss: 0.1387 - accuracy: 0.9539 - val_loss: 0.1904 - val_accuracy: 0.9300
Epoch 4/20
432/432 [==============================] - 251s 581ms/step - loss: 0.0954 - accuracy: 0.9685 - val_loss: 0.2168 - val_accuracy: 0.9283
Epoch 5/20
432/432 [==============================] - 251s 581ms/step - loss: 0.0640 - accuracy: 0.9811 - val_loss: 0.2836 - val_accuracy: 0.9238


In [19]:
x_test = tokenizer.texts_to_sequences(df_test['text'].values)
x_test = keras.utils.pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')
pred_test = model.predict(x_test)

288/288 [==============================] - 26s 90ms/step


In [20]:
pred_test = [0 if x[0]>x[1] else 1 for x in pred_test]

In [21]:
from sklearn.metrics import f1_score, confusion_matrix
test_f1 = f1_score(df_test['label'].values, pred_test)
test_cm = confusion_matrix(df_test['label'].values, pred_test)
test_tn, test_fp, test_fn, test_tp = confusion_matrix(df_test['label'].values, pred_test).ravel()

In [22]:
test_f1

0.9310089020771515

In [23]:
test_tn, test_fp, test_fn, test_tp

(4882, 292, 266, 3765)

# Testing with another dataset

In [24]:
# Load the dataset
dataset = load_dataset("mpingale/mental-health-chat-dataset")
dataset

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views', 'text'],
        num_rows: 2775
    })
})

In [ ]:
df1 = pd.DataFrame({
    'text': dataset['train']['questionText'],
    'label': dataset['train']['topic']
})